In [1]:
# Mount Google Drive
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"
print('Root path:', ROOT)                 

drive.mount(ROOT)       

Root path: /content/drive
Mounted at /content/drive


In [2]:
# %cd 'drive/My Drive/Waterloo/School/3B/MSCI 541'    # Change directory to the location defined in project_path
%cd drive/My Drive/Waterloo/School/3B/MSCI 541
%pwd

/content/drive/My Drive/Waterloo/School/3B/MSCI 541


'/content/drive/My Drive/Waterloo/School/3B/MSCI 541'

In [3]:
!pip install transformers datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 6.7 MB/s 
     |████████████████████████████████| 451 kB 76.8 MB/s 
     |████████████████████████████████| 7.6 MB 37.4 MB/s 
     |████████████████████████████████| 182 kB 83.4 MB/s 
     |████████████████████████████████| 132 kB 81.2 MB/s 
     |████████████████████████████████| 212 kB 76.5 MB/s 
     |████████████████████████████████| 127 kB 84.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [20]:
import pickle

docsPath = '/content/docs_for_hw5.pkl'
docs = pickle.load(open(docsPath,'rb'))
print(docs[0])

queriesPath = '/content/queries_for_hw5.pkl'
queries = pickle.load(open(queriesPath, 'rb'))

KOHL BACKS DOWN ON POLISH BORDER DEMANDS Chancellor Helmut Kohl, whose statements on Poland's border triggered international unease about German unification and a crisis in his Cabinet, today backed down from a demand that a border treaty be linked to Polish concessions. Kohl's government agreed to introduce a resolution in Parliament saying a united Germany should sign a treaty promising that it will never lay claim to Polish land, said the chancellor's chief of staff, Rudolf Seiters. The proposed resolution, to be introduced Thursday, is the closest the West German government has come to assuring Poland that a united Germany would never seek to recover land lost after the 1945 defeat of the Third Reich. Kohl has been reluctant to make such a statement, apparently fearing that he would lose the conservative vote in December elections. He has said he cannot speak for a united Germany. His recent statements on the border issue have stirred unease in Poland, the Soviet Union, the United 

In [21]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
from datasets import Dataset

tokenizer = AutoTokenizer.from_pretrained("castorini/monobert-large-msmarco")
model = AutoModelForSequenceClassification.from_pretrained("castorini/monobert-large-msmarco").to("cuda")

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--castorini--monobert-large-msmarco/snapshots/0a97706f3827389da43b83348d5d18c9d53876fa/config.json
Model config BertConfig {
  "_name_or_path": "castorini/monobert-large-msmarco",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--castorini--monobert-large-msmarco/snapshots/0a97706f3827389da43b83348d5d18c9d53876fa/vocab.txt
loading file tokenizer.json from cache at None
loa

In [22]:
import torch
# with torch.no_grad():
#   out = model(**tokens)
#   monobert_score = out.logits.log_softmax(-1)[0, -1].item()

args = TrainingArguments(output_dir="temp", per_device_eval_batch_size=32)
tokens = tokenizer(queries, docs, max_length=512, truncation="only_second")
dataset = Dataset.from_dict(tokens)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
trainer = Trainer(model, args, train_dataset=dataset, data_collator=data_collator, tokenizer=tokenizer)

predictions = trainer.predict(dataset)
scores = torch.tensor(predictions.predictions).log_softmax(-1)[:, -1]

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
***** Running Prediction *****
  Num examples = 4444
  Batch size = 32
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [24]:
list_of_scores = scores.tolist()
print(list_of_scores)
pickle.dump(list_of_scores,open('monobert_scores.pkl', 'wb'))

[-4.843038082122803, -8.108881950378418, -9.276004791259766, -0.028257273137569427, -9.090492248535156, -10.901639938354492, -8.438279151916504, -6.760483264923096, -9.722061157226562, -9.643165588378906, -9.100386619567871, -7.963621139526367, -8.834689140319824, -0.022462811321020126, -9.602362632751465, -8.492965698242188, -10.535223007202148, -5.989200592041016, -10.80236530303955, -9.210062980651855, -10.659497261047363, -9.27125072479248, -9.207588195800781, -8.368063926696777, -9.889277458190918, -11.256932258605957, -8.695822715759277, -9.430906295776367, -5.782552242279053, -10.974848747253418, -10.404789924621582, -8.097541809082031, -8.743985176086426, -6.644518852233887, -10.584976196289062, -10.075984954833984, -8.56584358215332, -9.277179718017578, -10.046295166015625, -11.40706729888916, -10.264114379882812, -10.09675121307373, -8.224459648132324, -8.909942626953125, -10.915181159973145, -5.9808549880981445, -8.227569580078125, -11.494147300720215, -9.972811698913574, -9